# CPD

In [1]:
import sys
sys.path.append('../src')
from read import *
from model import *
from train import *
from metric import *
device = 'cuda:0'

In [2]:
data = 'lastfm_time'
data_path = '../data'

cfg = DotMap()
cfg.name = data
cfg.dpath = data_path
cfg.opath = '../output'
cfg.unfair = 0.05
cfg.bs = 1024
cfg.random = 1
cfg.device = device
tensor = TensorDataset(cfg=cfg, path=cfg.dpath, name=cfg.name)
cfg.sizes = tensor.sizes

***********************************************************************************
[1] Read lastfm_time self...
[2] Read metadata...
[3] No normalization; values are already binary...
[4] Split the tensor into training/validation/test
 [4 - 1] Sparsify the minority group to make it more unfair
[5] Make statistics of group information
[6] Change the date type into torch
[7] Read lastfm_time tensor done...!
Tensor      || ['user', 'artist', 'time']; value
NNZ         || [861, 3066, 1586]; 76727 | 14311 | 14311
Sens. Attr  || user, gender: maj(['M']) min(['F'])
Entity      || Majority: 493 Minority: 368
NNZ         || Majority: [74740] Minority: [1987]
***********************************************************************************


In [3]:
cfg.rank = 10
cfg.lr = 0.001
cfg.wd = 0.001
cfg.n_iters = 10000
verbose = True
tensor.load_data()

In [4]:
model = CPD(cfg).to(cfg.device)
model

CPD(
  (factors): ParameterList(
      (0): Parameter containing: [torch.float32 of size 861x10 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 3066x10 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 1586x10 (GPU 0)]
  )
)

In [5]:
trainer = Trainer(model, tensor, cfg, wandb=None)
trainer.train()

Iters:   1 || training loss: 25.18951	Train RMSE: 0.48776 Valid RMSE: 0.48770	
Iters:   2 || training loss: 15.87554	Train RMSE: 0.44483 Valid RMSE: 0.45532	
Iters:   3 || training loss: 15.30129	Train RMSE: 0.46173 Valid RMSE: 0.48883	
Iters:   4 || training loss: 17.20089	Train RMSE: 0.49595 Valid RMSE: 0.53718	
Iters:   5 || training loss: 19.91678	Train RMSE: 0.53354 Valid RMSE: 0.58439	
Iters:   6 || training loss: 22.90641	Train RMSE: 0.57020 Valid RMSE: 0.62676	
Iters:   7 || training loss: 25.94309	Train RMSE: 0.60451 Valid RMSE: 0.66375	
Iters:   8 || training loss: 28.91367	Train RMSE: 0.63588 Valid RMSE: 0.69568	


In [6]:
res = evaluate_model(model, tensor)
print(f"MSE : {res['test_rmse'] * res['test_rmse']:.4f}")
print(f"MADE: {abs(res['MAD_Error']):.5f} Error1 : {res['Group0_Error']:.5f} Error2: {res['Group1_Error']:.5f}")

Test NRE: 0.4927 Test RMSE: 0.4572
***********************************************************************************
Calculate group fairness...
***********************************************************************************
MSE : 0.2091
MADE: 0.03280 Error1 : 0.19742 Error2: 0.23021
